In [3]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


In [22]:
cd /content/drive/My Drive/fininsight/Kobert_ner/pytorch-bert-crf-ner/

/content/drive/My Drive/fininsight/Kobert_ner/pytorch-bert-crf-ner


In [8]:
ls

'=0.1.6'    data_utils/                   LICENSE
'=0.4.0'    download.sh                   metric.py
'=0.6.0'    evaluate.py                   model/
'=1.5.0'    experiments/                  ptr_lm_model/
 app.py     inference.py                  README.md
 assets/    Interactive_shell_NER.ipynb   templates/
 bertviz/   kobert/                       train_bert_crf.py
 data_in/   legacy/                       Visualization_BERT_NER.ipynb


In [0]:
! pip install torch torchvision
! pip install pytorch_pretrained_bert>=0.4.0
! pip install mxnet>=1.5.0
! pip install gluonnlp #>=0.6.0
! pip install sentencepiece>=0.1.6
! pip install git+https://github.com/kmkurn/pytorch-crf#egg=pytorch_crf
! pip install transformers
! pip install tb-nightly
! pip install future

In [0]:
! pip install konlpy

In [0]:
from __future__ import absolute_import, division, print_function, unicode_literals
import json
import pickle
import torch
from gluonnlp.data import SentencepieceTokenizer
from model.net import KobertCRF
from data_utils.utils import Config
from data_utils.vocab_tokenizer import Tokenizer
from data_utils.pad_sequence import keras_pad_fn
from pathlib import Path

In [0]:
class DecoderFromNamedEntitySequence():
    def __init__(self, tokenizer, index_to_ner):
        self.tokenizer = tokenizer
        self.index_to_ner = index_to_ner

    def __call__(self, list_of_input_ids, list_of_pred_ids):
        input_token = self.tokenizer.decode_token_ids(list_of_input_ids)[0]
        pred_ner_tag = [self.index_to_ner[pred_id] for pred_id in list_of_pred_ids[0]]

        # ----------------------------- parsing list_of_ner_word ----------------------------- #
        list_of_ner_word = []
        entity_word, entity_tag, prev_entity_tag = "", "", ""
        for i, pred_ner_tag_str in enumerate(pred_ner_tag):
            if "B-" in pred_ner_tag_str:
                entity_tag = pred_ner_tag_str[-3:]

                if prev_entity_tag != entity_tag and prev_entity_tag != "":
                    list_of_ner_word.append({"word": entity_word.replace("▁", " "), "tag": prev_entity_tag, "prob": None})

                entity_word = input_token[i]
                prev_entity_tag = entity_tag
            elif "I-"+entity_tag in pred_ner_tag_str:
                entity_word += input_token[i]
            else:
                if entity_word != "" and entity_tag != "":
                    list_of_ner_word.append({"word":entity_word.replace("▁", " "), "tag":entity_tag, "prob":None})
                entity_word, entity_tag, prev_entity_tag = "", "", ""


        # ----------------------------- parsing decoding_ner_sentence ----------------------------- #
        decoding_ner_sentence = ""
        is_prev_entity = False
        prev_entity_tag = ""
        is_there_B_before_I = False

        for i, (token_str, pred_ner_tag_str) in enumerate(zip(input_token, pred_ner_tag)):
            if i == 0 or i == len(pred_ner_tag)-1: # remove [CLS], [SEP]
                continue
            token_str = token_str.replace('▁', ' ')  # '▁' 토큰을 띄어쓰기로 교체

            if 'B-' in pred_ner_tag_str:
                if is_prev_entity is True:
                    decoding_ner_sentence += ':' + prev_entity_tag+ '>'

                if token_str[0] == ' ':
                    token_str = list(token_str)
                    token_str[0] = ' <'
                    token_str = ''.join(token_str)
                    decoding_ner_sentence += token_str
                else:
                    decoding_ner_sentence += '<' + token_str
                is_prev_entity = True
                prev_entity_tag = pred_ner_tag_str[-3:] # 첫번째 예측을 기준으로 하겠음
                is_there_B_before_I = True

            elif 'I-' in pred_ner_tag_str:
                decoding_ner_sentence += token_str

                if is_there_B_before_I is True: # I가 나오기전에 B가 있어야하도록 체크
                    is_prev_entity = True
            else:
                if is_prev_entity is True:
                    decoding_ner_sentence += ':' + prev_entity_tag+ '>' + token_str
                    is_prev_entity = False
                    is_there_B_before_I = False
                else:
                    decoding_ner_sentence += token_str

        return list_of_ner_word, decoding_ner_sentence

In [0]:
! pip install pytorch-transformers

In [0]:
! python train_bert_crf.py

/usr/local/lib/python3.6/dist-packages/konlpy/tag/_okt.py:16: UserWarning: "Twitter" has changed to "Okt" since KoNLPy v0.4.5.
  warn('"Twitter" has changed to "Okt" since KoNLPy v0.4.5.')
using cached model
[██████████████████████████████████████████████████]
using cached model
len(token_to_idx):  8002
num of files:  1425
num of files:  2
using cached model
using cached model
num of train: 23032, num of val: 931
Epoch:   0% 0/15 [00:00<?, ?it/s]
Iteration:   0% 0/90 [00:00<?, ?it/s]epoch : 1, global_step : 1, tr_loss: 24537.520, tr_acc: 5.78%

Iteration:   1% 1/90 [01:15<1:51:40, 75.28s/it]epoch : 1, global_step : 2, tr_loss: 22135.842, tr_acc: 63.53%

Iteration:   2% 2/90 [02:25<1:48:03, 73.68s/it]epoch : 1, global_step : 3, tr_loss: 20525.904, tr_acc: 66.71%

Iteration:   3% 3/90 [03:34<1:44:51, 72.32s/it]epoch : 1, global_step : 4, tr_loss: 19261.059, tr_acc: 67.00%

Iteration:   4% 4/90 [04:43<1:42:15, 71.34s/it]epoch : 1, global_step : 5, tr_loss: 18320.475, tr_acc: 65.62%

Itera

In [0]:
def main(x):
    model_dir = Path('./experiments/base_model_with_crf')
    # /content/drive/My Drive/fininsight/Kobert_ner/pytorch-bert-crf-ner/experiments
    model_config = Config(json_path=model_dir / 'config.json') 
    

    # load vocab & tokenizer 
    tok_path = "./ptr_lm_model/tokenizer_78b3253a26.model" 
    ptr_tokenizer = SentencepieceTokenizer(tok_path)

    with open(model_dir / "vocab.pkl", 'rb') as f:
        vocab = pickle.load(f)
    tokenizer = Tokenizer(vocab=vocab, split_fn=ptr_tokenizer, pad_fn=keras_pad_fn, maxlen=model_config.maxlen)

    # load ner_to_index.json
    with open(model_dir / "ner_to_index.json", 'rb') as f:
        ner_to_index = json.load(f)
        index_to_ner = {v: k for k, v in ner_to_index.items()}

    # model
    model = KobertCRF(config=model_config, num_classes=len(ner_to_index), vocab=vocab)

    # load
    model_dict = model.state_dict()
    checkpoint = torch.load("./experiments/base_model_with_crf/best-epoch-16-step-1500-acc-0.993.bin", map_location=torch.device('cpu'))
    # checkpoint = torch.load("./experiments/base_model_with_crf_val/best-epoch-12-step-1000-acc-0.960.bin", map_location=torch.device('cpu'))
    convert_keys = {}
    for k, v in checkpoint['model_state_dict'].items():
        new_key_name = k.replace("module.", '')
        if new_key_name not in model_dict:
            print("{} is not int model_dict".format(new_key_name))
            continue
        convert_keys[new_key_name] = v

    model.load_state_dict(convert_keys)
    model.eval()
    device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')
    model.to(device)
    decoder_from_res = DecoderFromNamedEntitySequence(tokenizer=tokenizer, index_to_ner=index_to_ner)

    ner_text = []
    for text in x:
        input_text = text
        list_of_input_ids = tokenizer.list_of_string_to_list_of_cls_sep_token_ids([input_text])
        x_input = torch.tensor(list_of_input_ids).long()
        list_of_pred_ids = model(x_input)

        list_of_ner_word, decoding_ner_sentence = decoder_from_res(list_of_input_ids=list_of_input_ids, list_of_pred_ids=list_of_pred_ids)
        ner_text.append(decoding_ner_sentence)

    return ner_text

In [0]:
import re

def pre_fx(text):
      # 이메일 등장하면, 그 뒷문장 모두 삭제 
      text = re.sub("[a-zA-Z0-9]+\@[a-zA-Z0-9]+\.[a-z]{1,3}.[a-z]{1,3}.+",'',text).strip()
      text = re.sub('\(.+?연합뉴스\)','',text).strip() # (서울=연합뉴스)
      text = re.sub('\(.+?연합인포맥스\)','',text).strip()
      text = re.sub('\(.+?이데일리\)','',text).strip()
      text = re.sub('\(.+?조선일보\)','',text).strip()
      text = re.sub('\(.+?뉴시스\)',' ',text).strip()
      text = re.sub('\(.+?뉴스1\)','',text).strip()
      text = re.sub('\(.+?SBS\)','',text).strip()
      text = re.sub('\(.+?오마이뉴스\)','',text).strip()
      text = re.sub('\(.+?중앙일보\)','',text).strip()
      text = re.sub('\(.+?매일경제\)','',text).strip()
      text = re.sub('\(.+?문화일보\)','',text).strip()
      text = re.sub('\(.+?세계일보\)','',text).strip()
      text = re.sub('\(.+?머니투데이\)','',text).strip()
      text = re.sub('\(.+?서울경제\)','',text).strip()
      text = re.sub('\(.+?데일리안\)','',text).strip()
      text = re.sub('\(.+?KBS\)','',text).strip()
      text = re.sub('\(.+?MBN\)','',text).strip()
      text = re.sub('\(.+?YTN\)','',text).strip()
      text = re.sub('\(.+?프레시안\)','',text).strip()
      text = re.sub('\(.+?디지털타임즈\)','',text).strip()
      text = re.sub('\(.+?국민일보(과거)\)','',text).strip()
      text = re.sub('\(.+?국민일보\)','',text).strip()
      text = re.sub('\(.+?헤드럴경제\)','',text).strip()
      text = re.sub('\(.+?한국일보\)','',text).strip()
      text = re.sub('\(.+?아이뉴스24\)','',text).strip()
      text = re.sub('\(.+?노컷뉴스\)','',text).strip()
      text = re.sub('\(.+?연합뉴스TV\)','',text).strip()
      text = re.sub('\(.+?서울일보\)','',text).strip()
      text = re.sub('\(.+?동아일보\)','',text).strip()
      text = re.sub('\(.+?한국경제\)','',text).strip()
      text = re.sub('\(.+?미디어오늘(과거)\)','',text).strip()
      text = re.sub('\(.+?미디어오늘\)','',text).strip()
      text = re.sub('\(.+?조세일보\)','',text).strip()
      text = re.sub('\(.+?파이낸셜뉴스\)','',text).strip()
      text = re.sub('\(.+?경향신문\)','',text).strip()
      text = re.sub('\(.+?채널A\)','',text).strip()
      text = re.sub('\(.+?머니S\)','',text).strip()
      text = re.sub('\(.+?TVCHOSUN\)','',text).strip()
      text = re.sub('\(.+?한겨례\)','',text).strip()
      text = re.sub('\(.+?전자신문\)','',text).strip()
      text = re.sub('\(.+?SBSCNBC\)','',text).strip()
      text = re.sub('\(.+?한국경제TV\)','',text).strip()
      text = re.sub('\(.+?조선비즈\)','',text).strip()
      text = re.sub('\(.+?ZDNetKorea\)','',text).strip()
      text = re.sub('\[.+?\]','',text).strip()
      text = re.sub('\(사진.+?\)','',text).strip()
      # 신문사 이름 삭제 
      text = re.sub('조선일보|뉴시스|아시아경제|이데일리|뉴스1|SBS|오마이뉴스|중앙일보|매일경제|문화일보|세계일보|머니투데이|서울경제|데일리안|KBS|MBN|YTN|프레시안|디지털타임스|국민일보(과거)|국민일보|헤럴드경제|한국일보|아이뉴스24|노컷뉴스|연합뉴스TV|서울일보|동아일보|한국경제|미디어오늘(과거)|미디어오늘|조세일보|파이낸셜뉴스|경향신문|채널A|머니S|TVCHOSUN|한겨례|전자신문|SBSCNBC|한국경제TV|조선비즈|ZDNetKorea','',text).strip()
      # 000 기자, 000  가자, 000기자 삭제
      text = re.sub('[ㄱ-힑]+ 기자|[ㄱ-힑]+  기자|[ㄱ-힑]+기자','',text).strip()    
      text = re.sub('[a-zA-Z0-9]{1,20}\@[a-zA-Z0-9]{1,20}\.[a-z]{1,20}.+','',text).strip()    
      # 특수기호 삭제
      text = re.sub('[^a-zA-Z0-9ㄱ-힗.,”“"% ]','',text).strip()
      text = re.sub('[0-9]{1,4}.[0-9]{1,2}.[0-9]{1,2}.','',text).strip()
      text = re.sub('무단 전재 및 재배포 금지','',text).strip()
      text = re.sub(r'[\n\r\t]', '', str(text))
      text = re.sub('[^ ㄱ-ㅣ가-힣 0-9 A-Z a-z]+', '', str(text))
      return text

In [0]:
df_hwang = pd.read_csv('/content/drive/My Drive/fininsight/21대 총선분석/TexkRank/Data/황교안_pre.csv')

In [0]:
# df_hwang = pd.read_csv('/content/drive/My Drive/fininsight/21대 총선분석/TexkRank/Data/황교안_pre.csv')
df_hwang['content_clean'] = df_hwang['content'].apply(pre_fx)
df_hwang['content_ner'] = df_hwang['content_clean'].apply(main)
df_hwang.head()

In [0]:
df_hwang.to_csv('/content/drive/My Drive/fininsight/21대 총선분석/TexkRank/Data/황교안_ner.csv', index=False)

In [0]:
df_lee = pd.read_csv('/content/drive/My Drive/fininsight/21대 총선분석/TexkRank/Data/이낙연_pre.csv')
df_lee['content_clean'] = df_lee['content'].apply(pre_fx)
df_lee['content_ner'] = df_lee['content'].apply(main)
df_lee.to_csv('/content/drive/My Drive/fininsight/21대 총선분석/TexkRank/Data/이낙연_ner.csv', index=False)

In [0]:
df_jongro = pd.read_csv('/content/drive/My Drive/fininsight/21대 총선분석/TexkRank/Data/종로구_pre.csv')
df_jongro['content_clean'] = df_jongro['content'].apply(pre_fx)
df_jongro['content_ner'] = df_jongro['content'].apply(main)
df_jongro.to_csv('/content/drive/My Drive/fininsight/21대 총선분석/TexkRank/Data/종로구_ner.csv', index=False)